In [5]:
#Logging
import logging
import paramiko
import sys
import shutil
import time
import posixpath
import datetime
import os
import smtplib, getpass

#Working with email

smtp_object = smtplib.SMTP("smtp.gmail.com", 587)
smtp_object.ehlo()
smtp_object.starttls()
smtp_object.ehlo()
email = "learningandcertifications@gmail.com"
password = "amjutefntketxvfw"
smtp_object.login(email,password)
from_address = email
to_address = email
subject = "Reporting"


#Logging
get_date = datetime.datetime.now().strftime("%Y%m%d_%H%M")

logging.basicConfig(level=logging.ERROR, filename=f"/Users/homebase/Desktop/Python/Udemy Course/Milestone Projects/OPUS/Processing/Log/OutboundError/{get_date}-errorlog.log", filemode="w", format="%(asctime)s - %(levelname)s - %(message)s")

#Outbound Process

#path variables
local_path = os.path.join(os.getcwd(), 'Processing/Outbound/Reprocess')
remote = '/Files/OPUS/'

#Paramiko instance
client = paramiko.SSHClient()
client.load_system_host_keys()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

with open(f"Processing/Log/Outbound/{get_date}_outbound_log", 'a') as f:
    f.write(f"============= {get_date} Outbound Processing =============\n")
    try:
        client.connect(hostname = '192.168.64.3', username = 'testuser', password = 'TempPassword1')
        f.write("Connection successful\n")
        message = "SFTP Connection Successful"

        try:
            with client.open_sftp() as sftp:
                remote_location = sftp.listdir(remote) #os.listdir()
                for file in remote_location:
                    if file.endswith(".pdf"):
                        source = posixpath.join(remote, file)
                        download_path = os.path.join(local_path, file)
                        sftp.get(source, download_path)
                        f.write(f"{os.path.basename(source)} downloaded to {os.path.basename(local_path)}\n")
        
                        sftp.remove(source) #Delete
                        f.write(f"{os.path.basename(source)} deleted\n")
                        #message = "File handling successful"
        except Exception as e:
            logging.exception(f"Error handling {file}: {e}")
            message = f"Error handling {file}"
            sys.exit() #to stop the script if an error occurs
    except Exception as e:
        logging.exception(f"Error during SFTP connection: {e}")
        message = f"Error during SFTP connection"
        sys.exit() #to stop the script if an error occurs
    finally:
        client.close()
        msg = "Subject: "+subject+'\n'+message
        smtp_object.sendmail(from_address,to_address,msg)
        smtp_object.quit()